# Imports

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import f1_score
import random
import json
import time
import os
import datetime
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
import torch
import pickle
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.nn.parallel import DistributedDataParallel
from sklearn.metrics import f1_score
import h3

# Tokenization à partit du dataset nettoyé 
### (ie sans les trajets trop courts)

On commence par charger le dataset0

In [ ]:
with open('/home/daril_kw/data/02.06.23/train_clean.json', 'r') as openfile:

    # Reading from json file
    json_loaded = json.load(openfile)

print("We put the data in a dataset.")
 

#we put them in a dataframe
data_format = pd.DataFrame(data=json_loaded)

On fait la tokenization correcte (ordre latitude, longitude)

In [ ]:
data_format['Tokenization_2'] = data_format['POLYLINE'].apply(lambda x: [h3.geo_to_h3(x[i][0], x[i][1], 10) for i in range(len(x))])

#on enregistre dans un json le dataframe
data_format.to_json('/home/daril_kw/data/data_2_without_time_info.json',orient='records')

# Gestion du tokenizer

Le but est d'ajouter au tokenizer les jetons géographiques et de contexte.

### Extraction des données temporelles du timestamp

In [ ]:
#on commence par lire le json

with open('/home/daril_kw/data/data_2_without_time_info.json', 'r') as openfile:
    # Reading from json file
    data_format = json.load(openfile)

data_format = pd.DataFrame(data=data_format)



On ajoute les colonnes de contexte temporel.

In [ ]:
#on récupère la date à partir du timestamp
data_format['DATE'] = data_format['TIMESTAMP'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

#a partir de cela on récupère le jour de la semaine sous forme d'un nombre entre 1 et 7 pour lundi à dimanche
data_format['DAY'] = data_format['DATE'].apply(lambda x: str(datetime.datetime.strptime(x.split(' ')[0],'%Y-%m-%d').isocalendar()[2]))

#ensuite, on récupère l'heure sous forme d'un nombre entre 0 et 23
data_format['HOUR'] = data_format['DATE'].apply(lambda x: x.split(' ')[1].split(':')[0])

#enfin on recupère le numéro de la semaine dans l'année
data_format['WEEK'] = data_format['DATE'].apply(lambda x: str(datetime.datetime.strptime(x.split(' ')[0],'%Y-%m-%d').isocalendar()[1]))


On enlève les colonnes inutiles.

In [ ]:
data_clean.drop(['MISSING_DATA','DATE','ORIGIN_CALL','TRIP_ID', 'DAY_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'Nb_points', 'TIMESTAMP' ],axis=1,inplace=True)

On met dans le bon format les autres données de contexte.

In [ ]:
data_format['CALL_TYPE'] = data_format['CALL_TYPE'].apply(lambda x: str(1) if x=='A' else str(2) if x=='B' else str(3))

In [ ]:
if type(data_format['TAXI_ID'][0])!=str:
    data_format['TAXI_ID']=data_format['TAXI_ID'].apply(lambda x: str(x))
#idem pour le call_type
if type(data_format['CALL_TYPE'][0])!=str:
    data_format['CALL_TYPE']=data_format['CALL_TYPE'].apply(lambda x: str(x))

In [ ]:
#On sauvegarde le dataframe
data_format.to_json('/home/daril_kw/data/data_with_time_info_ok.json',orient='records')

### On ajoute au vocabulaires tous les tokens dont on a besoin.

In [ ]:
#on charge le tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

On commence par s'occuper des tokens géographiques issus de la tokenization h3.

In [ ]:
liste_token_geo = []

for i in range(len(data_format)):
    for j in range(len(data_format['Tokenization_2'][i])):
        liste_token_geo.append(data_format['Tokenization_2'][i][j])

#on enlève les doublons
liste_token_geo = list(set(liste_token_geo))

#on en profite pour sauvegarder la liste des tokens géographiques dans un fichier txt
with open('/home/daril_kw/data/liste_token_geo.txt', 'w') as f:
    for item in liste_token_geo:
        f.write("%s\n" % item)

#on garde le nombre de tokens géographiques pour la suite
nb_token_geo = len(liste_token_geo)


In [ ]:
#On ajoute les tokens géographiques au tokenizer
tokenizer.add_tokens(liste_token_geo)

On s'occupe maintenant des tokens de contexte.

In [ ]:
contextual_info_token = []

for i in range(len(data_format)):
    contextual_info_token.append(data_format['CALL_TYPE'][i])
    contextual_info_token.append(data_format['TAXI_ID'][i])
    contextual_info_token.append(data_format['DAY'][i])
    contextual_info_token.append(data_format['HOUR'][i])
    contextual_info_token.append(data_format['WEEK'][i])

#on enlève les doublons
contextual_info_token = list(set(contextual_info_token))

tokenizer.add_tokens(contextual_info_token)


On sauvegarde le tokenizer.

In [ ]:
tokenizer.save_pretrained('/home/daril_kw/data/tokenizer_final')

# Gestion du modèle

On charge le modèle.

In [ ]:
#On a besoin du nombre de labels, celui-ci correspond au nombre de tokens géographiques + 1 (pour le token [SEQ] indiquant la fin de la séquence)

nb_labels = nb_token_geo + 1

model=BertForSequenceClassification.from_pretrained("bert-base-cased",num_labels=nb_labels)


On adapte la taille de l'embedding au tokenizer.

In [ ]:
#on adapte la taille de l'embedding pour qu'elle corresponde au nombre de tokens géographiques + 1
model.resize_token_embeddings(len(tokenizer))

On sauvegarde le modèle.

In [ ]:
#On sauvegarde le modèle
model.save_pretrained('/home/daril_kw/data/model_final')

# Gestion de l'input (formattage)

Pour une tâche de classification, nous avons besoin que l'entrée soit sous une forme particulière. 

In [ ]:
with open('/home/daril_kw/data//home/daril_kw/data/data_with_time_info_ok.json', 'r') as openfile:
    json_loaded = json.load(openfile)
    
data_format = pd.DataFrame(data=json_loaded)

In [ ]:
#on ajoute un espace devant les éléments de la colonne HOUR, WEEK, CALL_TYPE, TAXI_ID en prévision de la concaténation pour séparer d'un espace les éléments de la colonne Tokenization et les autres colonnes

data_format['HOUR']=data_format['HOUR'].apply(lambda x: ' '+x)
data_format['WEEK']=data_format['WEEK'].apply(lambda x: ' '+x)
data_format['CALL_TYPE']=data_format['CALL_TYPE'].apply(lambda x: ' '+x)
data_format['TAXI_ID']=data_format['TAXI_ID'].apply(lambda x: ' '+x)
data_format['DAY']=data_format['DAY'].apply(lambda x: ' '+x)

In [ ]:
# la colonne CONTEXT_INPUT sera la concaténation du jour de la semaine, de l'heure et de la semaien de l'année pui de la colonne CALL_TYPE, de la colonne TAXI_ID, d'un espace et du dernier token de la colonne Tokenization
data_format['CONTEXT_INPUT'] =data_format['Tokenization_2'].apply(lambda x: x[-1]) + data_format['DAY'] + data_format['HOUR'] + data_format['WEEK'] + data_format['CALL_TYPE'] + data_format['TAXI_ID']


In [ ]:
#on récupère le nombre d'informations dans la colonne CONTEXT_INPUT
#Comme cette colonne contiient les informations en string séparé par un espace, on récupère la liste correspondante puis on compte le nombre d'éléments de cette liste
len_context_info = len(data_format['CONTEXT_INPUT'][0].split(' '))

#la colonne DEB_TRAJ sera la colonne Tokenization jusqu'a l'avant-dernier token exclu

data_format['DEB_TRAJ']=data_format['Tokenization_2'].apply(lambda x: x[:-2])


# on gère la longueur de la colonne CONTEXT_INPUT pour qu'après la concaténation, elle ne dépasse pas 512 tokens
#le -2 correspond aux deux tokens spéciaux [CLS] et [SEP]
# for exemple here, if the trajectory input is too long, we keep the 512-6-2=504 last tokens
data_format['DEB_TRAJ']=data_format['DEB_TRAJ'].apply(lambda x: x[-(512-len_context_info-2):] if len(x)>512-len_context_info-2 else x)

#then we keep the column in form of a string
data_format['DEB_TRAJ']=data_format['DEB_TRAJ'].apply(lambda x: ' '.join(x))


In [ ]:
data_format['TARGET']=data_format['Tokenization_2'].apply(lambda x: x[-2])


In [ ]:
#on enlève les colonnes inutiles
if 'Tokenization' in data_format.columns:
    data_format.drop(['Tokenization'],axis=1,inplace=True)
if 'CALL_TYPE' in data_format.columns:
    data_format.drop(['CALL_TYPE'],axis=1,inplace=True)
if 'TAXI_ID' in data_format.columns:
    data_format.drop(['TAXI_ID'],axis=1,inplace=True)
if 'DAY' in data_format.columns:
    data_format.drop(['DAY'],axis=1,inplace=True)
if 'HOUR' in data_format.columns:
    data_format.drop(['HOUR'],axis=1,inplace=True)
if 'WEEK' in data_format.columns:
    data_format.drop(['WEEK'],axis=1,inplace=True)
if 'Nb_points_token' in data_format.columns:
    data_format.drop(['Nb_points_token'],axis=1,inplace=True)


In [ ]:
#on sauvegarde le fichier au format json
data_format.to_json('/home/daril_kw/data/data_formatted_final.json',orient='records')

### Concaténation, padding et ajout tokens spéciaux

In [ ]:
c_inputs=data_format.CONTEXT_INPUT.values
traj_inputs=data_format.DEB_TRAJ.values
targets=data_format.TARGET.values

Gestion de l'entrée : 

In [ ]:
input_ids = []
full_inputs = []
attention_masks = []
for i in tqdm(range(len(c_inputs))):
    #no truncation is needed because we managed it before

    #we concatenate the context input and the trajectory input adding manually the CLS token and the SEP token
    full_input = '[CLS] ' + c_inputs[i] + ' ' + traj_inputs[i] + ' [SEP]'
    full_inputs.append(full_input)
    #encoded_c_input=tokenizer.encode(c_inputs[i], add_special_tokens=False)
    #encoded_traj_input=tokenizer.encode(traj_inputs[i], add_special_tokens=False)
    #we add manually the CLS token and the SEP token when we concatenate the two inputs
    #encoded_full_input=[101] + encoded_c_input + encoded_traj_input + [102]
    #the[101] token is the CLS token and the [102] token is the SEP token

    encoded_full_input=tokenizer.encode(full_input, add_special_tokens=False)
    #we pad the input to the maximum length of 512
    encoded_full_input=encoded_full_input + [0]*(512-len(encoded_full_input))
    input_ids.append(encoded_full_input)
    #we create the attention mask
    att_mask = [float(i>0) for i in encoded_full_input]
    attention_masks.append(att_mask)
    #the attention mask is a list of 0 and 1, 0 for the padded tokens and 1 for the other tokens
    #the float(i>0) is 0 if i=0 (ie if the token is a padded token) and 1 if i>0 (ie if the token is not a padded token)
    

Gestion des targets :

Le problème est que pour les labels, nous devons entrer des entiers entre 0 et len(targets)-1.

Pour cela, on  crée un dictionnaire qui associe à chaque label un entier entre 0 et len(targets)-1.


In [ ]:
targets_dict={}
for i in range(len(targets)):
    if targets[i] not in targets_dict:
        targets_dict[targets[i]]=len(targets_dict)

targets_input=[targets_dict[targets[i]] for i in range(len(targets))]

#le dictionnaire est sauvegardé au format json
with open('/home/daril_kw/data/targets_dict.json', 'w') as fp:
    json.dump(targets_dict, fp)

# le dictionnaire s'utilisera comme suit : 

# soit un entier 'target' non compris entre 0 et len(targets_dict)-1
# on récupère la valeur correspondante dans le dictionnaire avec targets_dict[target]
# inversement, si on a un entier 'target_encoded' compris entre 0 et len(targets_dict)-1
# on récupère la clé correspondante dans le dictionnaire avec list(targets_dict.keys())[target_encoded]

#la liste targets_input contient les targets du dataset encodées avec le dictionnaire targets_dict ie dans leur nouvel espace
#save in pickle the targets

with open('/home/daril_kw/data/targets_input.pkl', 'wb') as f:
    pickle.dump(targets_input, f)

In [ ]:
#on separe les données en train, validation et test

train_data, test_input, train_targets, test_targets = train_test_split(input_ids, targets_input,random_state=2023, test_size=0.2) 
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_data, train_targets,random_state=2023, test_size=0.1)

train_masks, test_masks, _, _ = train_test_split(attention_masks, targets_input,random_state=2023, test_size=0.2)
#train_masks, test_masks, _, _ = train_test_split(attention_masks, targets_input,random_state=2023, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(train_masks, train_targets,random_state=2023, test_size=0.1)


#on convertit les données en tenseurs
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
test_inputs = torch.tensor(test_input)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
test_labels = torch.tensor(test_targets)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
test_masks = torch.tensor(test_masks)




Dataloading :

In [ ]:
batch_size = 32

# Create the DataLoader for our training set, one for validation set and one for test set

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler, batch_size=batch_size)

prediction_data = TensorDataset(test_inputs, test_masks, test_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data,sampler=prediction_sampler, batch_size=batch_size)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained("/home/daril_kw/data/model_final",num_labels=nb_labels)
model.to(device)
model = DistributedDataParallel(model)


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5,eps = 1e-8)

# Number of training epochs. The BERT authors recommend between 2 and 4.
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps = 0,num_training_steps = total_steps)


In [ ]:
#on définit les fonctions utiles pour l'entrainement


#la focntion
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def flat_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,pred_flat,average='macro')



In [ ]:
seed_val = 2023
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Training

In [ ]:
#we store the loss and accuracy of each epoch
loss_values = []
accuracy_values = []
f1_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader),elapsed))
        

        """
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)"""

        batch = tuple(t.to(device) for t in batch)
  
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch


        #we set the gradients to zero
        model.zero_grad()
        #we make the forward pass
        outputs = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
        #we get the loss
        loss = outputs[0]
        #we accumulate the loss
        total_loss += loss.item()
        #we make the backward pass
        loss.backward()
        #we clip the gradient to avoid exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        #we update the parameters
        optimizer.step()
        #we update the learning rate
        scheduler.step()
    # Calculate the average loss over all of the batches.  
    avg_train_loss = total_loss / len(train_dataloader)
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently during evaluation.
    model.eval()
    # Tracking variables
    eval_loss, eval_accuracy,eval_f1 = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        #we unpack the batch
        b_input_ids, b_input_mask, b_labels = batch
        #we don't compute the gradient
        with torch.no_grad():
            #we make the forward pass
            outputs = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask)
            #we get the logits
            logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        #we compute the accuracy
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        #we compute the f1 score
        tmp_eval_f1 = flat_f1(logits, label_ids)
        #we accumulate the accuracy
        eval_accuracy += tmp_eval_accuracy
        #we accumulate the f1 score
        eval_f1 += tmp_eval_f1
        #we accumulate the number of examples
        nb_eval_examples += b_input_ids.size(0)
        #we accumulate the number of steps
        nb_eval_steps += 1
    #we compute the accuracy
    eval_accuracy = eval_accuracy / nb_eval_examples
    #we compute the f1 score
    eval_f1 = eval_f1 / nb_eval_examples
    print("  Accuracy: {0:.2f}".format(eval_accuracy))
    print("  F1 score: {0:.2f}".format(eval_f1))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    #we store the accuracy
    accuracy_values.append(eval_accuracy)
    #we store the f1 score
    f1_values.append(eval_f1)
print("")
print("Training complete!")

#in the trainning loop, the loss is computed for each batch, so we have to compute the average loss for each epoch
#the loss function is the cross entropy loss that is to say the negative log likelihood loss




In [ ]:
#we save the model
output_dir = './model_save/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print("Saving model to %s" % output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
#we save the loss and accuracy values
np.save(output_dir+'loss_values.npy',loss_values)
np.save(output_dir+'accuracy_values.npy',accuracy_values)

# Test

In [ ]:
model.eval()
 
# Tracking variables 
predictions , true_labels, list_inputs_test = [], [], []
print("We predict")
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
   
  # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

  # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

  #Store the inputs

    list_inputs_test.append(b_input_ids.tolist())

print('    DONE.')

In [ ]:
matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

pred_label= []
# For each input batch...
for i in range(len(true_labels)):

  # The predictions for this batch are a 2-column ndarray (one column for "0"
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  pred_label.append(pred_labels_i)
  # Calculate and store the coef for this batch.
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)
  matthews_set.append(matthews)


# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]


# Combine the inputs for each batch into a single list.
flat_list_inputs_test = [item for sublist in list_inputs_test for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

In [ ]:
# #display the confusion matrix
#cm = confusion_matrix(flat_true_labels, flat_predictions)
#print(cm)

In [ ]:
#
#we begin with the correct trajectory
#we get the input ids
correct_input_ids = [ [] for i in range(len(flat_list_inputs_test))]
#in the input, the part that interests us is the part after the cls token and after the six context tokens (so the last 512-len_context_info-1 tokens)

#we remove the padding tokens in the list flat_list_inputs_test
for i in range(len(flat_list_inputs_test)):
    for j in range(len(flat_list_inputs_test[i])):
        if flat_list_inputs_test[i][j] == 0:
            flat_list_inputs_test[i] = flat_list_inputs_test[i][:j]
            break


for i in range(len(flat_list_inputs_test)):
    correct_input_ids[i].append(flat_list_inputs_test[i][7:len(flat_list_inputs_test[i])-1])

    #the 7 is because we have the cls token and the six context tokens
    #the len(list_inputs_test[i][0])-1 is because we have the sep token at the end
    #we flatten

    correct_input_ids[i] = [item for sublist in correct_input_ids[i] for item in sublist]

    #we get the label that is to say the penultimate point of the trajectory
    correct_input_ids[i].append(list(targets_dict.keys())[flat_true_labels[i]])

    #we get the destination ie the last point of the trajectory that is to say the first token after the cls token in the input
    correct_input_ids[i].append(flat_list_inputs_test[i][1])




#we get the same thing but with the predicted labels instead of the true labels
incorrect_input_ids = [ [] for i in range(len(flat_list_inputs_test))]
for i in range(len(flat_list_inputs_test)):
    
    incorrect_input_ids[i].append(flat_list_inputs_test[i][7:len(flat_list_inputs_test[i])-1])

    #we don't want the list format so we flatten the list
    incorrect_input_ids[i] = [item for sublist in incorrect_input_ids[i] for item in sublist]

    #we have to use the dict to get the corresponding label
    incorrect_input_ids[i].append(list(targets_dict.keys())[flat_predictions[i]])
    incorrect_input_ids[i].append(flat_list_inputs_test[i][1])
    

#we have numbers but we want to get the corresponding words
#we get the words

#we do that for the incorrect trajectories
incorrect_words = []
for i in range(len(incorrect_input_ids)):
    incorrect_words.append([])
    for j in range(len(incorrect_input_ids[i])):
        incorrect_words[i].append(tokenizer.convert_ids_to_tokens(incorrect_input_ids[i][j]))
        #incorrect_words[i][j] is the jth word of the ith trajectory



#correct trajectories
correct_words = []
for i in range(len(correct_input_ids)):
    correct_words.append([])
    for j in range(len(correct_input_ids[i])):
        correct_words[i].append(tokenizer.convert_ids_to_tokens(correct_input_ids[i][j]))
        #correct_words[i][j] is the jth word of the ith trajectory


In [ ]:
import folium
import h3

reso = 10

map_center = [41.157943, -8.629105] # Porto coordinates
m = folium.Map(location=map_center, zoom_start=12)

h3_tokens= []
for i in range(len(correct_words)):
    for j in range(len(correct_words[i])):
        h3_tokens.append(correct_words[i][j])

h3_tokens= list(set(h3_tokens))

hexagon_vertices = []
for h3_token in h3_tokens:
    hexagon_vertices.append(h3.h3_to_geo_boundary(h3_token))

for i in range(len(hexagon_vertices)): 
    folium.Polygon(locations=hexagon_vertices[i],color='green',fill=True,fill_color='green',fill_opacity=0.4).add_to(m)

    


In [ ]:
#display the trajectories
#for that we have du detokenize with h3_to_geo from the h3 library

#the trajectories are in the correct_words list
for traj in correct_words:
    #we get the coordinates of the trajectory
    coords = []
    for token in traj:
        coords.append(h3.h3_to_geo(token))
    folium.PolyLine(coords, color="blue", weight=2.5, opacity=1).add_to(m)

    for i in range(len(traj)):
        folium.CircleMarker(traj[i], radius=1, color='blue', fill=True, fill_color='blue', fill_opacity=0.6).add_to(m)





In [ ]:
#display the map
m